# 📥 Ingestão e Mapeamento das Tabelas

## ✅ Objetivo do Notebook

Este notebook executa as seguintes tarefas principais:

---

### 1. 🔍 Comparação entre Datasets de Mesmo Tipo

Verifica se estruturas de **tabelas semelhantes podem ser empilhadas**, garantindo a consistência do schema entre tabelas fragmentadas.

---

### 2. 📝 Criação de Arquivo `.json` com a Descrição das Tabelas

Gera automaticamente um **arquivo com metadados estruturais** de cada tabela, incluindo nomes de colunas, tipos de dados, totais e presença de chave primária.

---

### 3. 📖 Geração de Dicionário de Variáveis (em Português)

Cria um **mapeamento interpretável das variáveis**, facilitando a compreensão dos nomes técnicos e tornando a análise mais acessível.


### Criação da SparkSession

In [1]:
# Criação da SparkSession
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ExemploSparkSession") \
    .getOrCreate()

### Importações

In [67]:
from pyspark.sql.functions import col, when, count, isnan, isnull, countDistinct
import json
import pandas as pd
import os
import glob

In [71]:
diretorio = r"C:\Users\fred\meu_projeto_etl"

### 🔍Comparação entre datasets do mesmo tipo

In [21]:
## Carregamento das tabelas necessárias para validação de empilhamento
caminho = r"C:\Users\fred\meu_projeto_etl\data\raw\train"
train_base = spark.read.parquet(f"{caminho}\\train_base.parquet")
train_applprev_1_0 = spark.read.parquet(f"{caminho}\\train_applprev_1_0.parquet")
train_applprev_1_1 = spark.read.parquet(f"{caminho}\\train_applprev_1_1.parquet")
train_credit_bureau_a_1_0 = spark.read.parquet(f"{caminho}\\train_credit_bureau_a_1_0.parquet")
train_credit_bureau_a_1_1 = spark.read.parquet(f"{caminho}\\train_credit_bureau_a_1_1.parquet")
train_credit_bureau_a_2_0 = spark.read.parquet(f"{caminho}\\train_credit_bureau_a_2_0.parquet")
train_credit_bureau_b_1 = spark.read.parquet(f"{caminho}\\train_credit_bureau_b_1.parquet")
train_credit_bureau_b_2 = spark.read.parquet(f"{caminho}\\train_credit_bureau_b_2.parquet")
train_debitcard_1 = spark.read.parquet(f"{caminho}\\train_debitcard_1.parquet")
train_deposit_1 = spark.read.parquet(f"{caminho}\\train_deposit_1.parquet")
train_other_1 = spark.read.parquet(f"{caminho}\\train_other_1.parquet")
train_person_1 = spark.read.parquet(f"{caminho}\\train_person_1.parquet")
train_person_2 = spark.read.parquet(f"{caminho}\\train_person_2.parquet")
train_static_0_0 = spark.read.parquet(f"{caminho}\\train_static_0_0.parquet")
train_static_0_1 = spark.read.parquet(f"{caminho}\\train_static_0_1.parquet")
train_stati_cb_0 = spark.read.parquet(f"{caminho}\\train_static_cb_0.parquet")
train_tax_registry_a_1 = spark.read.parquet(f"{caminho}\\train_tax_registry_a_1.parquet")
train_tax_registry_b_1 = spark.read.parquet(f"{caminho}\\train_tax_registry_b_1.parquet")
train_tax_registry_c_1 = spark.read.parquet(f"{caminho}\\train_tax_registry_c_1.parquet")
train_credit_bureau_a_2_1 = spark.read.parquet(f"{caminho}\\train_credit_bureau_a_2_1.parquet")
train_credit_bureau_a_2_5 = spark.read.parquet(f"{caminho}\\train_credit_bureau_a_2_5.parquet")
train_credit_bureau_a_2_10= spark.read.parquet(f"{caminho}\\train_credit_bureau_a_2_10.parquet")

In [10]:
## Função para avaliar tabelas com colunas iguais
def comparar_tabelas(tabela1, tabela2):
    colunas1 = set(tabela1)
    colunas2 = set(tabela2)
    
    if colunas1 == colunas2:
        print("✅ Colunas iguais (ignora ordem).")
    else:
        print("✅ Colunas diferentes (ignora ordem).")

In [22]:
comparar_tabelas(train_applprev_1_0.columns, train_applprev_1_1.columns)

✅ Colunas iguais (ignora ordem).


In [7]:
comparar_tabelas(train_applprev_1_0.columns, train_applprev_1_1.columns)

✅ Colunas iguais (ignora ordem).


In [8]:
comparar_tabelas(train_credit_bureau_a_1_0.columns, train_credit_bureau_a_1_1.columns)

✅ Colunas iguais (ignora ordem).


In [11]:
comparar_tabelas(train_credit_bureau_a_1_0.columns, train_credit_bureau_a_2_0.columns)

✅ Colunas diferentes (ignora ordem).


In [12]:
comparar_tabelas(train_static_0_0.columns, train_static_0_1.columns)

✅ Colunas iguais (ignora ordem).


In [13]:
comparar_tabelas(train_tax_registry_a_1.columns, train_tax_registry_b_1.columns)

✅ Colunas diferentes (ignora ordem).


In [14]:
comparar_tabelas(train_credit_bureau_a_2_0.columns, train_credit_bureau_a_2_10.columns)

✅ Colunas iguais (ignora ordem).


## 📊 Resultado das Comparações
### 🧩 Estrutura dos Nomes dos Datasets

A nomenclatura dos arquivos segue um padrão que permite identificar suas subdivisões em grupos, com a seguinte estrutura:

(treino_ou_teste)_(fonte_de_dados)_(grupo_de_variáveis)(subgrupo)


treino_ou_teste: Indica se o dataset pertence ao conjunto de treino (train) ou teste (test).

fonte_de_dados: Define a origem da informação (por exemplo: credit_bureau, applprev, tax_registry, static).

grupo_de_variáveis: Agrupa variáveis com características semelhantes ou função analítica similar.

subgrupo (opcional): Representa uma subdivisão ainda mais específica, quando aplicável.

### 🧱 Possibilidade de Empilhamento

Os datasets que compartilham o mesmo nome até a parte de <grupo_de_variáveis> podem ser empilhados (concatenados) de forma segura, pois representam a mesma estrutura sem conflito semântico. Essa estratégia é útil para consolidação e análise integrada entre diferentes clientes ou aplicações.

## 📝 Criação de arquivo .json com a descrição das tabelas

In [64]:
# Caminho base dos arquivos
caminho_base = f"{diretorio}\\data\raw\train"

# Lista dos grupos esperados
grupos = [
    "train_applprev_1",
    "train_applprev_2",
    "train_credit_bureau_a_1",
    "train_credit_bureau_a_2",
    "train_credit_bureau_b_1",
    "train_credit_bureau_b_2",
    "train_static_0",
    "train_base",
    "train_debitcard_1",
    "train_deposit_1",
    "train_other_1",
    "train_person_1",
    "train_person_2",
    "train_static_cb_0",
    "train_tax_registry_a_1",
    "train_tax_registry_b_1",
    "train_tax_registry_c_1"
]


✅ train_applprev_1 carregado com 2 arquivo(s).
✅ train_applprev_2 carregado com 1 arquivo(s).
✅ train_credit_bureau_a_1 carregado com 4 arquivo(s).
✅ train_credit_bureau_a_2 carregado com 11 arquivo(s).
✅ train_credit_bureau_b_1 carregado com 1 arquivo(s).
✅ train_credit_bureau_b_2 carregado com 1 arquivo(s).
✅ train_static_0 carregado com 2 arquivo(s).
✅ train_base carregado com 1 arquivo(s).
✅ train_debitcard_1 carregado com 1 arquivo(s).
✅ train_deposit_1 carregado com 1 arquivo(s).
✅ train_other_1 carregado com 1 arquivo(s).
✅ train_person_1 carregado com 1 arquivo(s).
✅ train_person_2 carregado com 1 arquivo(s).
✅ train_static_cb_0 carregado com 1 arquivo(s).
✅ train_tax_registry_a_1 carregado com 1 arquivo(s).
✅ train_tax_registry_b_1 carregado com 1 arquivo(s).
✅ train_tax_registry_c_1 carregado com 1 arquivo(s).

📦 Variáveis carregadas:
 - train_applprev_1
 - train_applprev_2
 - train_credit_bureau_a_1
 - train_credit_bureau_a_2
 - train_credit_bureau_b_1
 - train_credit_bureau

### Empilhamento de tabelas

In [ ]:
# Função para carregar e empilhar arquivos de um mesmo grupo
def carregar_e_empilhar(grupo):
    padrao = os.path.join(caminho_base, f"{grupo}_*.parquet")
    arquivos = sorted(glob.glob(padrao))

    # Também tenta carregar arquivo único exato (sem sufixo)
    arquivo_unico = os.path.join(caminho_base, f"{grupo}.parquet")
    if os.path.exists(arquivo_unico):
        arquivos.append(arquivo_unico)

    if not arquivos:
        print(f"🚫 Nenhum arquivo encontrado para {grupo}")
        return None

    colunas_padrao = None
    dfs_validos = []

    for arquivo in arquivos:
        df = spark.read.parquet(arquivo)
        colunas_atuais = df.columns

        if colunas_padrao is None:
            colunas_padrao = colunas_atuais
            dfs_validos.append(df)
        elif colunas_atuais == colunas_padrao:
            dfs_validos.append(df)
        else:
            print(f"🚫 Colunas diferentes no arquivo: {arquivo}")
            print(f"    Esperado: {colunas_padrao}")
            print(f"    Encontrado: {colunas_atuais}")

    if not dfs_validos:
        print(f"⚠️ Nenhum DataFrame válido para {grupo}")
        return None

    # Empilhar os DataFrames (mesmas colunas)
    df_final = dfs_validos[0]
    for df in dfs_validos[1:]:
        df_final = df_final.unionByName(df)

    print(f"✅ {grupo} carregado com {len(dfs_validos)} arquivo(s).")
    return df_final

In [ ]:
# Dicionário com os DataFrames carregados com sucesso
variaveis_criadas = {}

# Carregar todos os grupos
for grupo in grupos:
    df = carregar_e_empilhar(grupo)
    if df is not None:
        variaveis_criadas[grupo] = df

# Exibir nomes dos DataFrames criados
print("\n📦 Variáveis carregadas:")
for nome in variaveis_criadas:
    print(f" - {nome}")

In [65]:
## Função para arquivo de descrição das tabelas
def analise_basica_spark_dict(df, nome_tabela, chave_primaria=None):
    schema = [(f.name, f.dataType.simpleString()) for f in df.schema.fields]
    n_regis = df.count()
    n_col = len(df.columns)
    print(f'Tabela: {nome_tabela}')
    print(f'Número de registros: {n_regis}')
    print(f'Número de colunas: {n_col}')
    describe_df = df.describe()
    print(describe_df)
    describe_data = {row["summary"]: {col: row[col] for col in row.asDict() if col != "summary"} for row in describe_df.collect()}

    nulls_expr = [
        ((count(when(col(c).isNull(), c)) / n_regis) * 100).alias(c)
        for c in df.columns
    ]
    nulls_df = df.select(nulls_expr)
    nulls_dict = nulls_df.collect()[0].asDict()
    nulls_filtrados = {k: round(v, 2) for k, v in nulls_dict.items() if v > 0}

    pk_status = "não verificada"
    if chave_primaria:
        if chave_primaria not in df.columns:
            pk_status = "coluna inexistente"
        else:
            distinct_count = df.select(chave_primaria).distinct().count()
            pk_status = "válida" if distinct_count == n_regis else "inválida"

    return {
        "tabela": nome_tabela,
        "total_registros": n_regis,
        "total_colunas": n_col,
        "schema": schema,
        "describe": describe_data,
        "colunas_com_nulos": nulls_filtrados,
        "chave_primaria": {
            "coluna": chave_primaria,
            "status": pk_status
        }
    }


In [68]:
# Acumulador de resultados
resultados = {}

# Alimentar resultados a cada execução
for nome, df in variaveis_criadas.items():
    resultado = analise_basica_spark_dict(df, nome_tabela=nome, chave_primaria="case_id")  
    resultados[nome] = resultado

Tabela: train_applprev_1
Número de registros: 6525979
Número de colunas: 41
DataFrame[summary: string, case_id: string, actualdpd_943P: string, annuity_853A: string, approvaldate_319D: string, byoccupationinc_3656910L: string, cancelreason_3545846M: string, childnum_21L: string, creationdate_885D: string, credacc_actualbalance_314A: string, credacc_credlmt_575A: string, credacc_maxhisbal_375A: string, credacc_minhisbal_90A: string, credacc_status_367L: string, credacc_transactions_402L: string, credamount_590A: string, credtype_587L: string, currdebt_94A: string, dateactivated_425D: string, district_544M: string, downpmt_134A: string, dtlastpmt_581D: string, dtlastpmtallstes_3545839D: string, education_1138M: string, employedfrom_700D: string, familystate_726L: string, firstnonzeroinstldate_307D: string, inittransactioncode_279L: string, mainoccupationinc_437A: string, maxdpdtolerance_577P: string, num_group1: string, outstandingdebt_522A: string, pmtnum_8L: string, postype_4733339M: s

Tabela: train_base
Número de registros: 1526659
Número de colunas: 5
DataFrame[summary: string, case_id: string, date_decision: string, MONTH: string, WEEK_NUM: string, target: string]
Tabela: train_debitcard_1
Número de registros: 157302
Número de colunas: 6
DataFrame[summary: string, case_id: string, last180dayaveragebalance_704A: string, last180dayturnover_1134A: string, last30dayturnover_651A: string, num_group1: string, openingdate_857D: string]
Tabela: train_deposit_1
Número de registros: 145086
Número de colunas: 5
DataFrame[summary: string, case_id: string, amount_416A: string, contractenddate_991D: string, num_group1: string, openingdate_313D: string]
Tabela: train_other_1
Número de registros: 51109
Número de colunas: 7
DataFrame[summary: string, case_id: string, amtdebitincoming_4809443A: string, amtdebitoutgoing_4809440A: string, amtdepositbalance_4809441A: string, amtdepositincoming_4809444A: string, amtdepositoutgoing_4809442A: string, num_group1: string]
Tabela: train_per

In [69]:
# Salvar arquivo json com informações da descrição das tabelas
with open(f"{diretorio}\\docs\analise_tabelas.json", "w", encoding="utf-8") as f:
    json.dump(resultados, f, ensure_ascii=False, indent=4)

In [70]:
# Carrega o JSON em um dicionário Python
with open(f"{diretorio}\\docs\analise_tabelas.json", "r", encoding="utf-8") as f:
    dados = json.load(f)

# Transformar o dicionário em uma lista de tabelas
dados_lista = list(dados.values())

# Criar DataFrame com o resumo das tabelas
df_resumo = pd.DataFrame(dados_lista)

from IPython.display import display

display(df_resumo.head(20))


,tabela,total_registros,total_colunas,schema,describe,colunas_com_nulos,chave_primaria
0,train_applprev_1,6525979,41,"[[case_id, bigint], [actualdpd_943P, double], ...","{'count': {'case_id': '6525979', 'actualdpd_94...","{'actualdpd_943P': 0.04, 'annuity_853A': 3.84,...","{'coluna': 'case_id', 'status': 'inválida'}"
1,train_applprev_2,14075487,6,"[[case_id, bigint], [cacccardblochreas_147M, s...","{'count': {'case_id': '14075487', 'cacccardblo...","{'cacccardblochreas_147M': 0.78, 'conts_type_5...","{'coluna': 'case_id', 'status': 'inválida'}"
2,train_credit_bureau_a_1,15940537,79,"[[case_id, bigint], [annualeffectiverate_199L,...","{'count': {'case_id': '15940537', 'annualeffec...","{'annualeffectiverate_199L': 95.44, 'annualeff...","{'coluna': 'case_id', 'status': 'inválida'}"
3,train_credit_bureau_a_2,188298452,19,"[[case_id, bigint], [collater_typofvalofguaran...","{'count': {'case_id': '188298452', 'collater_t...","{'collater_valueofguarantee_1124L': 98.54, 'co...","{'coluna': 'case_id', 'status': 'inválida'}"
4,train_credit_bureau_b_1,85791,45,"[[case_id, bigint], [amount_1115A, double], [c...","{'count': {'case_id': '85791', 'amount_1115A':...","{'amount_1115A': 49.08, 'contractdate_551D': 4...","{'coluna': 'case_id', 'status': 'inválida'}"
5,train_credit_bureau_b_2,1286755,6,"[[case_id, bigint], [num_group1, bigint], [num...","{'count': {'case_id': '1286755', 'num_group1':...","{'pmts_dpdvalue_108P': 0.42, 'pmts_pmtsoverdue...","{'coluna': 'case_id', 'status': 'inválida'}"
6,train_static_0,1526659,168,"[[case_id, bigint], [actualdpdtolerance_344P, ...","{'count': {'case_id': '1526659', 'actualdpdtol...","{'actualdpdtolerance_344P': 27.39, 'amtinstpai...","{'coluna': 'case_id', 'status': 'válida'}"
7,train_base,1526659,5,"[[case_id, bigint], [date_decision, string], [...","{'count': {'case_id': '1526659', 'date_decisio...",{},"{'coluna': 'case_id', 'status': 'válida'}"
8,train_debitcard_1,157302,6,"[[case_id, bigint], [last180dayaveragebalance_...","{'count': {'case_id': '157302', 'last180dayave...","{'last180dayaveragebalance_704A': 92.23, 'last...","{'coluna': 'case_id', 'status': 'inválida'}"
9,train_deposit_1,145086,5,"[[case_id, bigint], [amount_416A, double], [co...","{'count': {'case_id': '145086', 'amount_416A':...",{'contractenddate_991D': 54.92},"{'coluna': 'case_id', 'status': 'inválida'}"


## 📊 Resultado da descrição das tabelas
### 🧩 Estrutura da descrição

Baseado na estudo feito anteriormente, os datasets foram empilhados conforme os parâmetros estabelecidos no estudo gerando um panorâma geral das váriáveis de cada grupo característico de tabelas.

O arquivo resultante desse processo é um arquivo json. A escolha por esse formato foi mediante o grupo de informações que seria gerado. A estrutura do json se mostrou com melhor distribuição do que formatos tabulares.

A maioria das tabelas apresenta valores nulos em suas colunas o que aponta para avaliação da necessidade de tratamento ou descarte dessas variáveis.

A maioria das colunas não possuem chave primária (relacionamento 1:N). Mas possuem todas possuem a variável case_id. Essa variável será a chave primária após o tratamento necessário em cada tabela.

A tabela **train_base** será a tabela semente onde as outras tabelas, que se mostrarem agregadoras, serão agregadas para construção de uma tabela geral. 

## 📖 Geração de dicionário de variáveis (em português)

In [15]:
# Carregando dicionário de variáveies
descricao = spark.read.csv(f"{diretorio}\\data\raw\feature_definitions.csv", header = True)

In [16]:
# Transformar os itens da descrição de tabelas em lista e iterar por índice
estrutura_tabelas = []

tabelas_lista = list(tabelas.items())

for i in range(len(tabelas_lista)):
    nome_tabela, df = tabelas_lista[i]
    for campo in df.schema.fields:
        estrutura_tabelas.append((nome_tabela, campo.name, campo.dataType.simpleString()))

# Cria DataFrame com o índice de posição
estrutura_df = spark.createDataFrame(estrutura_tabelas, ["tabela", "coluna", "tipo_dado"])
estrutura_df.show(truncate=False)

+------------------+--------------------------+---------+
|tabela            |coluna                    |tipo_dado|
+------------------+--------------------------+---------+
|train_base        |case_id                   |bigint   |
|train_base        |date_decision             |string   |
|train_base        |MONTH                     |bigint   |
|train_base        |WEEK_NUM                  |bigint   |
|train_base        |target                    |bigint   |
|train_applprev_1_0|case_id                   |bigint   |
|train_applprev_1_0|actualdpd_943P            |double   |
|train_applprev_1_0|annuity_853A              |double   |
|train_applprev_1_0|approvaldate_319D         |string   |
|train_applprev_1_0|byoccupationinc_3656910L  |double   |
|train_applprev_1_0|cancelreason_3545846M     |string   |
|train_applprev_1_0|childnum_21L              |double   |
|train_applprev_1_0|creationdate_885D         |string   |
|train_applprev_1_0|credacc_actualbalance_314A|double   |
|train_applpre

In [17]:
# Junção da descrição das tabelas com o dicionário 
dicionario_variaveis = estrutura_df.join(descricao, estrutura_df.coluna == descricao.Variable, how='left')

In [18]:
dicionario_variaveis = dicionario_variaveis.drop('Variable')

In [19]:
dicionario_variaveis.show()

+------------------+--------------------+---------+--------------------+
|            tabela|              coluna|tipo_dado|         Description|
+------------------+--------------------+---------+--------------------+
|        train_base|             case_id|   bigint|                NULL|
|        train_base|       date_decision|   string|                NULL|
|        train_base|               MONTH|   bigint|                NULL|
|        train_base|            WEEK_NUM|   bigint|                NULL|
|        train_base|              target|   bigint|                NULL|
|train_applprev_1_0|             case_id|   bigint|                NULL|
|train_applprev_1_0|      actualdpd_943P|   double|Days Past Due (DP...|
|train_applprev_1_0|        annuity_853A|   double|Monthly annuity f...|
|train_applprev_1_0|   approvaldate_319D|   string|Approval Date of ...|
|train_applprev_1_0|byoccupationinc_3...|   double|Applicant's incom...|
|train_applprev_1_0|cancelreason_3545...|   string|

In [21]:
# Instalar pacate para tradução da coluna "Description"
!pip install deep-translator

In [22]:
# Tradução dos valores da coluna "Description"
from deep_translator import GoogleTranslator
from pyspark.sql.functions import col
import pandas as pd

# Extrair valores únicos da coluna Description (não nulos)
descricoes_unicas = (dicionario_variaveis
                     .select("Description")
                     .where(col("Description").isNotNull())
                     .distinct()
                     .rdd.flatMap(lambda x: x)
                     .collect())

# Traduzir usando GoogleTranslator
mapa_traducao = {desc: GoogleTranslator(source='en', target='pt').translate(desc) for desc in descricoes_unicas}

In [23]:
# Criação da coluna "Descricao"
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# UDF para mapear as traduções
@udf(StringType())
def traduzir_udf(desc):
    return mapa_traducao.get(desc, None)

# Criar nova coluna com as traduções
tabela_descritiva = dicionario_variaveis.withColumn("Descricao", traduzir_udf(col("Description")))

In [24]:
tabela_descritiva.select("coluna", "Description", "Descricao").show(truncate=False)

+--------------------------+------------------------------------------------------------------------------+-----------------------------------------------------------------------------+
|coluna                    |Description                                                                   |Descricao                                                                    |
+--------------------------+------------------------------------------------------------------------------+-----------------------------------------------------------------------------+
|case_id                   |NULL                                                                          |NULL                                                                         |
|date_decision             |NULL                                                                          |NULL                                                                         |
|MONTH                     |NULL                                      

In [25]:
# Transformação em arquivos Pandas para salvar em .md
tabela_descritiva_pd = tabela_descritiva.toPandas()

In [26]:
# Salvar como Markdown
md_path = f"{diretorio}\\docs\tabela_descritiva.md"


with open(md_path, "w", encoding="utf-8") as f:
    f.write(tabela_descritiva_pd.to_markdown(index=False))


## 📊 Resultado da Geração de dicionário de variáveis
### 🧩 Estrutura de execução

O dicionário gerado foi baseado no arquivo **feature_definitions.csv** disponibilizado na base de dados originalizados no kaggle.
A tradução da coluna **Description** foi realizada utilizando a biblioteca **deep-translator** gerando assim a variável **Descricao**
Após esse tratamento, foi gerado o arquivo **tabela_descritiva.md** encontrada em:
**f"{diretorio}\\data\raw\feature_definitions.csv"**